## define feature extraction functions

In [8]:
from scipy.signal import find_peaks, hilbert, welch
from scipy.fft import fft, fftfreq
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

def detect_peaks(signal):
    peaks_results = []
    median_height = np.median(signal)
    std_height = np.std(signal)
    height = median_height + std_height  # Setting height to be above median + 1 std deviation
    distance = len(signal) * 0.05  # Example to set distance to 5% of signal length
    prominence = std_height * 0.5  # Setting prominence to be half of std deviation
    
    # Detecting peaks
    peaks, properties = find_peaks(signal, height=height, distance=distance, prominence=prominence)
    
    peak_count = len(peaks)
    average_peak_height = np.mean(properties["peak_heights"]) if "peak_heights" in properties else 0
    average_distance = np.mean(np.diff(peaks)) if len(peaks) > 1 else 0
    average_prominence = np.mean(properties["prominences"]) if "prominences" in properties else 0

    peaks_results.append({
        "peak_count": peak_count,
        "average_peak_height": average_peak_height,
        "average_distance": average_distance,
        "average_prominence": average_prominence,
    })
    
    return peaks_results

def calculate_variance_std_dev(signals):
    variance = np.var(signals)
    std_dev = np.std(signals)
    return variance, std_dev

def calculate_rms(signals):
    rms = np.sqrt(np.mean(signals**2))
    return rms

def freq_bands(signals, fs=500):
    band_features = {}
    frequencies, psd = welch(signals, fs=fs, nperseg=min(32, len(signals)))
    bands = {'delta': (1, 4), 'theta': (4, 8), 'alpha': (8, 13), 'beta': (13, 30)}
    for name, (low, high) in bands.items():
        idx = np.logical_and(frequencies >= low, frequencies <= high)
        band_features[f'{name}_band_power'] = np.mean(psd[idx])
    return band_features, frequencies, psd

def calculate_spectral_entropy(signal, fs=500):
    frequencies, psd = welch(signal, fs=fs, nperseg=min(len(signal), fs * 2))
    normalized_psd = psd / np.sum(psd)
    spectral_entropy = -np.sum(normalized_psd * np.log2(normalized_psd))
    return spectral_entropy

def spectral_centroids(signals, fs=500):
    fft_result = fft(signals)
    frequencies = fftfreq(len(signals), 1.0/fs)
    magnitude = np.abs(fft_result)
    centroid = np.sum(frequencies * magnitude) / np.sum(magnitude)
    return centroid

def spectral_edge_density(signals, fs=500, percentage=95):
    fft_result = fft(signals)
    frequencies = fftfreq(len(signals), 1.0/fs)
    positive_frequencies = frequencies[frequencies >= 0]
    positive_fft_result = fft_result[frequencies >= 0]
    magnitude = np.abs(positive_fft_result)
    cumulative_sum = np.cumsum(np.sort(magnitude)[::-1])
    total_power = np.sum(magnitude)
    threshold = total_power * (percentage / 100)
    spectral_edge = positive_frequencies[np.argmax(cumulative_sum >= threshold)]
    return spectral_edge

def phase_locking_values(signals):
    num_signals = signals.shape[0]
    plv_matrix = np.zeros((num_signals, num_signals))
    for i in range(num_signals):
        for j in range(i+1, num_signals):  # Only compute for unique pairs
            signal1 = signals[i]
            signal2 = signals[j]
            analytic_signal1 = hilbert(signal1)
            analytic_signal2 = hilbert(signal2)
            phase1 = np.angle(analytic_signal1)
            phase2 = np.angle(analytic_signal2)
            phase_diff = phase1 - phase2
            plv = np.abs(np.sum(np.exp(1j * phase_diff)) / len(signal1))
            plv_matrix[i, j] = plv
            plv_matrix[j, i] = plv  # PLV is symmetric
    return plv_matrix

def calculate_higuchi_fractal_dimension(signals, k_max=10):
    hfd_values = []
    N = len(signals)
    L = []
    x = np.asarray(signals)
    for k in range(1, k_max + 1):
        Lk = []
        for m in range(k):
            Lkm_sum = 0
            max_index = int((N - m - 1) / k) + 1  # Ensures valid indexing
            for i in range(1, max_index):
                Lkm_sum += abs(x[m + i*k] - x[m + (i-1)*k])
            if max_index - 1 > 0:  # Check to avoid division by zero
                Lkm = Lkm_sum * (N - 1) / (k * (max_index - 1))
                Lk.append(Lkm)
            else:
                Lk.append(0)  # Or handle appropriately
        if np.mean(Lk) > 0:
            L.append(np.log(np.mean(Lk)))
        else:
            L.append(np.log(np.finfo(float).eps))  # Using machine epsilon for a small positive number
    if len(L) > 0 and np.all(np.isfinite(L)):
        hfd = np.polyfit(np.log(range(1, k_max + 1)), L, 1)[0]  # Linear fit to log-log plot
        hfd_values.append(hfd)
    else:
        hfd_values.append(np.nan)  # Append NaN or another placeholder to indicate failure
    return hfd_values

def calculate_zero_crossing_rate(signals):
    sign_changes = np.diff(np.sign(signals))
    zero_crossings = np.count_nonzero(sign_changes)
    zero_crossing_rate = zero_crossings / (len(signals) - 1)
    return zero_crossing_rate

def time_warping_factor(signals):
    average_signal = np.mean(np.array(signals), axis=0)
    warping_factors = []
    for signal in signals:
        signal = np.atleast_1d(signal)
        average_signal = np.atleast_1d(average_signal)
        distance, _ = fastdtw(np.squeeze(signal), np.squeeze(average_signal), dist=euclidean)
        warping_factors.append(distance)
    return warping_factors

def evolution_rate(signals):
    analytic_signal = hilbert(signals)
    envelope = np.abs(analytic_signal)
    derivative = np.diff(envelope)
    rate = np.mean(np.abs(derivative))
    return rate

def extract_features(signals, fs=500):
    features = {}
    
    for signal in signals:
        peaks_results = detect_peaks(signal)
        variance, std_dev = calculate_variance_std_dev(signal)
        rms = calculate_rms(signal)
        band_features, frequencies, psd = freq_bands(signal, fs)
        spectral_entropy = calculate_spectral_entropy(signal, fs)
        centroid = spectral_centroids(signal, fs)
        spectral_edge = spectral_edge_density(signal, fs)
        plv = phase_locking_values(signals)
        hfd_values = calculate_higuchi_fractal_dimension(signal)
        zero_crossing_rate = calculate_zero_crossing_rate(signal)
        warping_factors = time_warping_factor(signals)
        rate = evolution_rate(signal)
        
        features.update({
            "peak_counts": peaks_results[0]['peak_count'],
            "average_peak_height": peaks_results[0]['average_peak_height'],
            "average_distance": peaks_results[0]['average_distance'],
            "average_prominence": peaks_results[0]['average_prominence'],
            "variance": variance,
            "std_dev": std_dev,
            "rms": rms,
            "delta_band_power": band_features['delta_band_power'],
            "theta_band_power": band_features['theta_band_power'],
            "alpha_band_power": band_features['alpha_band_power'],
            "beta_band_power": band_features['beta_band_power'],
            "spectral_entropy": spectral_entropy,
            "centroids": centroid,
            "spectral_edge_density": spectral_edge,
            "higuchi_fractal_dimension": hfd_values[0],
            "zero_crossing_rate": zero_crossing_rate,
            "evolution_rate": rate
        })
    
    return features, frequencies, psd

def analyze_signals(buffer):
    signals = buffer.T  # Assuming signals are organized as channels x samples in the buffer
    
    features_list = []
    for i in range(signals.shape[0]):
        features, frequencies, psd = extract_features(signals[i])
        features_list.append(features)

    return features_list, frequencies, psd


## downsample neural data to 500hz, windowed at 25hz, save as npy, and extract features and save as npy

In [9]:
import os
import mne
import numpy as np
from scipy.signal import find_peaks, hilbert, welch
from scipy.fft import fft, fftfreq
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

# Path to your EDF file
file_path = '/home/vincent/datasets/raw/sub-01_ses-task_task-game_run-01_ieeg.edf'

# Load the EDF file
raw = mne.io.read_raw_edf(file_path, preload=True, stim_channel=None)

# Electrode names matched to centroids (retrieved from your earlier analysis)
matched_names = ['B21', 'A48', 'B56', 'A5', 'B57', 'B55', 'B4', 'B17', 'B18', 'A46', 'A9', 'A16', 'A18', 'A12', 'A39', 'A43', 'A13', 'B51', 'A38', 'B25', 'B37', 'B36', 'B44', 'B54', 'A34', 'A3', 'A24', 'A31', 'B48', 'B33', 'B40', 'B22']

# Function to generate the corresponding channel names with the "POL" prefix
def generate_pol_channel_name(name):
    if name.startswith('POL'):
        return name
    elif name.startswith('EEG'):
        return f'POL {name[4:]}'
    else:
        return f'POL {name}'  # Assuming other prefixes can be safely prefixed with "POL"

# Adjusted matched names with "POL" prefix
adjusted_matched_names = [generate_pol_channel_name(name) for name in matched_names]

print("Adjusted matched names:", adjusted_matched_names)

# Filter to keep only the indices of channels that are in adjusted_matched_names
indices = [raw.ch_names.index(name) for name in adjusted_matched_names if name in raw.ch_names]

print("Number of channels selected:", len(indices))  # Print the number of selected channels

# Extract data for these channels
selected_data, times = raw[indices, :]

# Downsample the data to 500 Hz if the original sampling rate is 1000 Hz
if raw.info['sfreq'] == 1000:
    raw.resample(500)

# Extract the downsampled data
selected_data, times = raw[indices, :]

# Function to segment the data
def segment_data(data, segment_length):
    num_segments = data.shape[1] // segment_length
    segmented_data = np.array(np.split(data[:, :num_segments * segment_length], num_segments, axis=1))
    return segmented_data

# Segment length for 0.25 seconds at 500 Hz
segment_length = 125

# Segment the data
segmented_data = segment_data(selected_data, segment_length)

print(f"Segmented data shape: {segmented_data.shape}")  # Should be (num_segments, 32, 125)

# Normalize the data to the range [0, 1]
def normalize_data(data):
    data_min = np.min(data, axis=(0, 2), keepdims=True)
    data_max = np.max(data, axis=(0, 2), keepdims=True)
    normalized_data = (data - data_min) / (data_max - data_min)
    return normalized_data

normalized_data = normalize_data(segmented_data)

# Function to extract features
def extract_features(signals, fs=500):
    features_list = []
    for signal in signals:
        features = {}
        
        # Peak detection
        peaks, properties = find_peaks(signal, height=np.median(signal) + np.std(signal), distance=len(signal) * 0.05, prominence=np.std(signal) * 0.5)
        features['peak_counts'] = len(peaks)
        features['average_peak_height'] = np.mean(properties['peak_heights']) if 'peak_heights' in properties else 0
        features['average_distance'] = np.mean(np.diff(peaks)) if len(peaks) > 1 else 0
        features['average_prominence'] = np.mean(properties['prominences']) if 'prominences' in properties else 0
        
        # Variance and standard deviation
        features['variance'] = np.var(signal)
        features['std_dev'] = np.std(signal)
        
        # RMS
        features['rms'] = np.sqrt(np.mean(signal**2))
        
        # Frequency bands and PSD
        frequencies, psd = welch(signal, fs=fs, nperseg=min(32, len(signal)))
        bands = {'delta': (1, 4), 'theta': (4, 8), 'alpha': (8, 13), 'beta': (13, 30)}
        for name, (low, high) in bands.items():
            idx = np.logical_and(frequencies >= low, frequencies <= high)
            features[f'{name}_band_power'] = np.mean(psd[idx])
        features['frequencies'] = frequencies
        features['psds'] = psd
        
        # Spectral entropy
        normalized_psd = psd / np.sum(psd)
        spectral_entropy = -np.sum(normalized_psd * np.log2(normalized_psd))
        features['spectral_entropy'] = spectral_entropy
        
        # FFT and spectral features
        fft_result = fft(signal)
        frequencies = fftfreq(len(signal), 1.0/fs)
        magnitude = np.abs(fft_result)
        features['fft_results'] = fft_result
        features['magnitudes'] = magnitude
        features['centroids'] = np.sum(frequencies * magnitude) / np.sum(magnitude)
        
        # Spectral edge density
        positive_frequencies = frequencies[frequencies >= 0]
        positive_fft_result = fft_result[frequencies >= 0]
        magnitude = np.abs(positive_fft_result)
        cumulative_sum = np.cumsum(np.sort(magnitude)[::-1])
        total_power = np.sum(magnitude)
        threshold = total_power * 0.95
        features['spectral_edge_density'] = positive_frequencies[np.argmax(cumulative_sum >= threshold)]
        features['positive_frequencies'] = positive_frequencies
        features['positive_fft_results'] = positive_fft_result
        features['cumulative_sums'] = cumulative_sum
        features['total_powers'] = total_power
        features['thresholds'] = threshold
        
        # Phases and phase locking values
        analytic_signal = hilbert(signal)
        features['phases'] = np.angle(analytic_signal)
        plv_matrix = phase_locking_values(signals)
        features['pairwise_phase_locking_values'] = plv_matrix
        
        # Higuchi Fractal Dimension
        features['higuchi_fractal_dimension'] = calculate_higuchi_fractal_dimension(signal, k_max=10)[0]
        
        # Zero Crossing Rate
        features['zero_crossing_rate'] = calculate_zero_crossing_rate(signal)
        
        # Evolution rate
        envelope = np.abs(analytic_signal)
        derivative = np.diff(envelope)
        features['evolution_rate'] = np.mean(np.abs(derivative))
        
        # IMFS (Placeholder, actual EMD implementation required)
        features['imfs'] = []  # Perform EMD to extract IMFs if required
        
        # Signal shapes and average signal shapes (Placeholder, actual shape extraction logic required)
        features['signal_shapes'] = signal.shape
        features['average_signal_shapes'] = np.mean(signal)
        
        # Warping factors (Placeholder, actual warping factor extraction logic required)
        features['warping_factors'] = time_warping_factor(signals)
        
        # Analytic signals and envelopes
        features['analytic_signals'] = analytic_signal
        features['envelops'] = envelope
        features['derivatives'] = derivative
        
        features_list.append(features)
    
    return features_list

# Extract features from each window
features_list = []
for window in normalized_data:
    features = extract_features(window)
    features_list.append(features)

# Ensure the directory exists
save_dir = '/home/vincent/datasets'
os.makedirs(save_dir, exist_ok=True)

# Save the normalized data and features
np.save(os.path.join(save_dir, 'normalized_neural_data.npy'), normalized_data)
np.save(os.path.join(save_dir, 'extracted_features.npy'), features_list)

print("Feature extraction complete. Data saved at /home/vincent/datasets.")


Extracting EDF parameters from /home/vincent/MySSD/JupyterProjects/AAA_projects/UnlimitedResearchCooperative/Synthetic_Intelligence_Labs/Bio-Silicon-Synergetic-Intelligence-System/Software/CNN_feature_extraction_project/datasets/raw/sub-01_ses-task_task-game_run-01_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1898115  =      0.000 ...  1898.115 secs...
Adjusted matched names: ['POL B21', 'POL A48', 'POL B56', 'POL A5', 'POL B57', 'POL B55', 'POL B4', 'POL B17', 'POL B18', 'POL A46', 'POL A9', 'POL A16', 'POL A18', 'POL A12', 'POL A39', 'POL A43', 'POL A13', 'POL B51', 'POL A38', 'POL B25', 'POL B37', 'POL B36', 'POL B44', 'POL B54', 'POL A34', 'POL A3', 'POL A24', 'POL A31', 'POL B48', 'POL B33', 'POL B40', 'POL B22']
Number of channels selected: 32
Segmented data shape: (7592, 32, 125)


/home/vincent/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/vincent/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Feature extraction complete. Data saved at /home/vincent/datasets.


In [1]:
import numpy as np

# Load the extracted features
features_path = '/home/vincent/datasets/extracted_features.npy'
features_list = np.load(features_path, allow_pickle=True)

# Calculate the number of segments per second (since each segment is 0.25 seconds)
segments_per_second = 4

# Get the number of segments for the first 5 seconds
num_segments = segments_per_second * 5

def print_feature(name, value):
    if isinstance(value, (list, np.ndarray)):
        print(f"{name}: {np.array(value).shape}")
        if len(value) <= 10:  # Print small arrays entirely
            print(value)
        else:
            print(f"{value[:10]}...")  # Print only the first 10 elements of large arrays
    elif isinstance(value, dict):
        print(f"{name}: dict with keys {list(value.keys())}")
    else:
        print(f"{name}: {value}")

# Print all features for the first 5 seconds of data
for i in range(num_segments):
    print(f"Features for segment {i + 1} (time window {i * 0.25} to {(i + 1) * 0.25} seconds):")
    features_array = features_list[i]
    if isinstance(features_array, np.ndarray):
        for features in features_array:
            if isinstance(features, dict):  # Ensure it's a dictionary
                for key, value in features.items():
                    print_feature(key, value)
            else:
                print(f"The feature is not in the expected dictionary format. Found type: {type(features)}")
    else:
        print(f"The feature is not in the expected dictionary format. Found type: {type(features_array)}")
    print("\n" + "-"*50 + "\n")


Features for segment 1 (time window 0.0 to 0.25 seconds):
peak_counts: 5
average_peak_height: 0.5079212106393244
average_distance: 14.75
average_prominence: 0.06723033045428184
variance: 0.0011257027138713545
std_dev: 0.033551493467077656
rms: 0.44976274712502184
delta_band_power: nan
theta_band_power: nan
alpha_band_power: nan
beta_band_power: 2.4896638203875245e-05
spectral_entropy: 2.4307362722310164
fft_results: (125,)
[56.06369508-0.j         -0.18520568+1.50435089j -0.45846848+0.46659581j
 -0.82139928-0.6311641j   0.666536  +0.89753252j -0.35861259+1.22257631j
  0.38786818+0.02222775j -0.12796837-0.19214113j  0.10029156-0.09601887j
 -0.26412714+0.26402626j]...
magnitudes: (125,)
[56.06369508  1.51570866  0.65414447  1.03588846  1.11796014  1.27408627
  0.38850456  0.23085519  0.13884531  0.37346086]...
centroids: -5.623450663373117e-16
spectral_edge_density: 80.0
higuchi_fractal_dimension: -0.587820394052086
zero_crossing_rate: 0.0
evolution_rate: 0.014795976347803264
peak_counts

In [ ]:
## create tensors for each feature

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split

# Load the extracted features
features_path = '/home/vincent/datasets/extracted_features.npy'
features_list = np.load(features_path, allow_pickle=True)

# Preprocess the features to create a unified tensor for each feature
def preprocess_features(features_list):
    all_features = []
    for segment in features_list:
        segment_features = {}
        for channel in segment:
            if isinstance(channel, dict):
                for key, value in channel.items():
                    if key not in segment_features:
                        segment_features[key] = []
                    if isinstance(value, np.ndarray):
                        segment_features[key].append(value)
                    else:
                        segment_features[key].append(np.array([value]))
        all_features.append(segment_features)
    
    # Convert lists to numpy arrays for each feature
    for key in all_features[0].keys():
        for i in range(len(all_features)):
            all_features[i][key] = np.stack(all_features[i][key])
    
    return all_features

# Convert the features to a tensor dictionary
features_dict = preprocess_features(features_list)
tensor_dict = {key: torch.tensor(np.array([seg[key] for seg in features_dict]), dtype=torch.float32) for key in features_dict[0].keys()}

# Assume we have corresponding labels (for illustration)
labels_array = np.random.randint(0, 2, size=(tensor_dict['peak_counts'].shape[0], tensor_dict['peak_counts'].shape[1]))
labels_tensor = torch.tensor(labels_array, dtype=torch.float32)

# Create a TensorDataset and DataLoader for each feature
datasets = {}
for key, tensor in tensor_dict.items():
    datasets[key] = TensorDataset(tensor, labels_tensor)

# Split each dataset into train, validation, and test sets
train_datasets = {}
val_datasets = {}
test_datasets = {}

for key, dataset in datasets.items():
    train_size = int(0.7 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    train_datasets[key], val_datasets[key], test_datasets[key] = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders for each feature
train_loaders = {key: DataLoader(ds, batch_size=32, shuffle=True) for key, ds in train_datasets.items()}
val_loaders = {key: DataLoader(ds, batch_size=32) for key, ds in val_datasets.items()}
test_loaders = {key: DataLoader(ds, batch_size=32) for key, ds in test_datasets.items()}

# Now you can use train_loaders, val_loaders, and test_loaders in your training loop
# For example, to access the DataLoader for 'peak_counts':
train_loader_peak_counts = train_loaders['peak_counts']
val_loader_peak_counts = val_loaders['peak_counts']
test_loader_peak_counts = test_loaders['peak_counts']

# Example of iterating through the DataLoader for 'peak_counts'
for inputs, labels in train_loader_peak_counts:
    # Your training loop here
    print("Inputs:", inputs)
    print("Labels:", labels)
    break  # Just to illustrate the first batch
